In [1]:
from astropy.coordinates import SkyCoord
from astroquery.utils.tap.core import TapPlus
import importlib
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.append('../../')

from astroLuSt.database_interface import (
    database_utils as aldidu,
    simbadinterface as aldisi
)

importlib.reload(aldidu)

#style for plotting
plt.style.use('astroLuSt.styles.LuSt_style')



2024-11-06 11:34:54.728791: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# `get_reference_objects()`
* searching fore closeby constant reference objects

In [2]:
importlib.reload(aldidu)

#get coordinates
SDI = aldisi.SimbadDatabaseInterface(n_jobs=1)
df_ids = SDI.get_ids(["RR Lyr", "TV Boo"])
coords = SkyCoord(df_ids["ra"], df_ids["dec"], unit="deg")

#extract potential reference ojects
res_tab1 = aldidu.get_reference_objects(coords=coords[0], radius=.05)
res_tab2 = aldidu.get_reference_objects(coords=coords[1], radius=.1)
display(res_tab1)
display(res_tab2)



oid,ra,dec,main_id,otype,ids,distance,vartyp_mesvar
,deg,deg,,,,,
int64,float64,float64,object,object,object,float64,object
13657430,291.40124541359,42.80931251755,Gaia DR3 2125984008083980928,*,Gaia DR3 2125984008083980928|Gaia DR2 2125984008083980928,0.03577733430113383,
12448530,291.41752353239997,42.76032540198,2MASS J19254020+4245371,*,Gaia DR3 2125983046017836544|TIC 159717459|2MASS J19254020+4245371|KIC 7199152|Gaia DR2 2125983046017836544,0.04462333254031259,


oid,ra,dec,main_id,otype,ids,distance,vartyp_mesvar
,deg,deg,,,,,
int64,float64,float64,object,object,object,float64,object
21037280,214.15861818394293,42.26231737542833,Gaia DR3 1492228976169235200,*,Gaia DR3 1492228976169235200|Gaia DR2 1492228976169235200,0.09771036592599006,


# `query_upload_table()`
* uploading a table in pieces to execute larger query

In [4]:
importlib.reload(aldidu)

df = pd.DataFrame(data={
    'id':[
        4295806720, 38655544960, 343597448960,
        549755818112, 828929527040, 1275606125952,
        1340029955712, 1374389600384, 1619203481984,
        1653563247744, 2199023259904, 2546916445184,
        2851858288640, 2920577765120, 2989297243776,
        3062312520192, 3332894779520, 3371550165888,
    ]
}, dtype=int)
query = f"""
    SELECT ut.*, sh.Source
    FROM tap_upload.upload_table AS ut INNER JOIN
        "I/354/starhorse2021" AS sh
            ON ut.id = sh.Source
"""


tap = TapPlus(
    url='http://tapvizier.cds.unistra.fr/TAPVizieR/tap',
    tap_context=None,
    verbose=True,
)

df_res = aldidu.query_upload_table(
    tap=tap, query=query,
    df_upload=df,
    upload_table_name='upload_table',
    query_async=False,  #no asynchronous query needed, as table is small
    nsplits=3,
    verbose=2,
    parallel_kwargs=dict(n_jobs=3, verbose=3),
    launch_job_kwargs=dict(verbose=False),
)

df_res

Created TAP+ (v20200428.1) - Connection:
	Host: tapvizier.cds.unistra.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443


/home/lukas/venvs/astroLuSt/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


KeyboardInterrupt: 